### Import das dependências

In [1]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### 1. Importar o dataset - "Caracterização de Pontos de Consumo (CPEs), com contratos ativos"


In [2]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

caracterizacao_pes_path = data_path + '20-caracterizacao-pes-contrato-ativo.csv'

df_caracterizacao_pes = pd.read_csv(caracterizacao_pes_path, delimiter=';')

### 2. Ver o dataset

In [3]:
df_caracterizacao_pes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393706 entries, 0 to 393705
Data columns (total 9 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Ano                           393706 non-null  int64 
 1   Mês                           393706 non-null  int64 
 2   CodDistritoConcelhoFreguesia  393706 non-null  object
 3   Distrito                      393706 non-null  object
 4   Concelho                      393706 non-null  object
 5   Freguesia                     393706 non-null  object
 6   Tipo de Instalação            393706 non-null  object
 7   Nível de Tensão               393706 non-null  object
 8   CPEs (#)                      393706 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 27.0+ MB


1. Verificar se há elementos null

In [4]:
# Verificar nulls
null_counts = df_caracterizacao_pes.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

df_caracterizacao_pes.head(15)

Número de nulls por coluna:
Ano                             0
Mês                             0
CodDistritoConcelhoFreguesia    0
Distrito                        0
Concelho                        0
Freguesia                       0
Tipo de Instalação              0
Nível de Tensão                 0
CPEs (#)                        0
dtype: int64


,Ano,Mês,CodDistritoConcelhoFreguesia,Distrito,Concelho,Freguesia,Tipo de Instalação,Nível de Tensão,CPEs (#)
0,2023,12,160626,Viana do Castelo,Ponte da Barca,"União das freguesias de Crasto, Ruivos e Grovelas",Não Doméstico,Baixa Tensão Especial,1
1,2023,12,010805,Aveiro,Estarreja,Pardilhó,Não Doméstico,Baixa Tensão Especial,4
2,2023,12,111603,Lisboa,Odivelas,Odivelas,Não Doméstico,"Muito Alta, Alta e Média Tensões",22
3,2023,3,170636,Vila Real,Montalegre,"União das freguesias de Cambeses do Rio, Donõe...",Não Doméstico,Baixa Tensão Normal,42
4,2023,3,011109,Aveiro,Mealhada,"União das freguesias da Mealhada, Ventosa do B...",Não Doméstico,"Muito Alta, Alta e Média Tensões",23
5,2024,3,061201,Coimbra,Pampilhosa da Serra,Cabril,Não Doméstico,Baixa Tensão Normal,46
6,2024,3,011109,Aveiro,Mealhada,"União das freguesias da Mealhada, Ventosa do B...",Não Doméstico,Baixa Tensão Normal,911
7,2024,3,121103,Portalegre,Monforte,Santo Aleixo,Não Doméstico,Baixa Tensão Normal,61
8,2024,3,181210,Viseu,Penedono,União das freguesias de Antas e Ourozinho,Não Doméstico,Baixa Tensão Normal,51
9,2023,7,160434,Viana do Castelo,Monção,União das freguesias de Anhões e Luzio,Iluminação Pública,Baixa Tensão Normal,4


### 3. Tratamento e agregação dos dados

1. Remover atributos como a Distrito, CodDistritoConcelhoFreguesia 
2. Agrupar por concelhos e remover a freguesia

1. Remover atributos como a Distrito, CodDistritoConcelhoFreguesia

In [5]:
# Remover atributos como o distrito e código distrito concelho freguesia
df_caracterizacao_pes.drop(columns=['Distrito', 'CodDistritoConcelhoFreguesia'], inplace=True)
df_caracterizacao_pes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393706 entries, 0 to 393705
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Ano                 393706 non-null  int64 
 1   Mês                 393706 non-null  int64 
 2   Concelho            393706 non-null  object
 3   Freguesia           393706 non-null  object
 4   Tipo de Instalação  393706 non-null  object
 5   Nível de Tensão     393706 non-null  object
 6   CPEs (#)            393706 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 21.0+ MB


2. Agrupar por concelhos e remover a freguesia

In [6]:
df_agrupado_caracterizacao = df_caracterizacao_pes.groupby(['Concelho', 'Tipo de Instalação','Nível de Tensão', 'Ano', 'Mês']).sum().reset_index()
df_agrupado_caracterizacao.drop(columns=['Freguesia'], inplace=True)
df_agrupado_caracterizacao.rename(columns={'Concelho': 'concelho', 'Tipo de Instalação':'tipo_de_instalacao', 'Nível de Tensão':'nivel_de_tensao', 'Ano':'ano', 'Mês':'mes', 'CPEs (#)':'cpes_quantidade'}, inplace=True)
df_agrupado_caracterizacao.head()

,concelho,tipo_de_instalacao,nivel_de_tensao,ano,mes,cpes_quantidade
0,Abrantes,Doméstico,Baixa Tensão Especial,2022,3,1
1,Abrantes,Doméstico,Baixa Tensão Especial,2022,4,1
2,Abrantes,Doméstico,Baixa Tensão Especial,2022,5,1
3,Abrantes,Doméstico,Baixa Tensão Especial,2022,6,1
4,Abrantes,Doméstico,Baixa Tensão Especial,2022,7,1


### 4. Verificação dos dados

1. Verificação número de linhas
2. Verificação dados

1. Verificação número de linhas

In [7]:
# Contagem de linhas antes do agrupamento
num_linhas_antes = len(df_caracterizacao_pes)

# Contagem de linhas depois do agrupamento
num_linhas_depois = len(df_agrupado_caracterizacao)

# Exibição das contagens
print("Número de linhas antes do agrupamento:", num_linhas_antes)
print("Número de linhas depois do agrupamento:", num_linhas_depois)

df_agrupado_caracterizacao.info()

df_agrupado_caracterizacao.head(15)

Número de linhas antes do agrupamento: 393706
Número de linhas depois do agrupamento: 48562
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48562 entries, 0 to 48561
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   concelho            48562 non-null  object
 1   tipo_de_instalacao  48562 non-null  object
 2   nivel_de_tensao     48562 non-null  object
 3   ano                 48562 non-null  int64 
 4   mes                 48562 non-null  int64 
 5   cpes_quantidade     48562 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.2+ MB


,concelho,tipo_de_instalacao,nivel_de_tensao,ano,mes,cpes_quantidade
0,Abrantes,Doméstico,Baixa Tensão Especial,2022,3,1
1,Abrantes,Doméstico,Baixa Tensão Especial,2022,4,1
2,Abrantes,Doméstico,Baixa Tensão Especial,2022,5,1
3,Abrantes,Doméstico,Baixa Tensão Especial,2022,6,1
4,Abrantes,Doméstico,Baixa Tensão Especial,2022,7,1
5,Abrantes,Doméstico,Baixa Tensão Especial,2022,8,1
6,Abrantes,Doméstico,Baixa Tensão Especial,2022,9,1
7,Abrantes,Doméstico,Baixa Tensão Especial,2022,10,1
8,Abrantes,Doméstico,Baixa Tensão Especial,2022,11,1
9,Abrantes,Doméstico,Baixa Tensão Especial,2022,12,1


2. Verificação de dados

In [8]:
# Para a coleção de Consumos Mensais por Concelho
df_consumos_lisboa = df_agrupado_caracterizacao[df_agrupado_caracterizacao['concelho'] == 'Lisboa']
df_consumos_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 230 entries, 19578 to 19807
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   concelho            230 non-null    object
 1   tipo_de_instalacao  230 non-null    object
 2   nivel_de_tensao     230 non-null    object
 3   ano                 230 non-null    int64 
 4   mes                 230 non-null    int64 
 5   cpes_quantidade     230 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 12.6+ KB


In [9]:
df_consumos_lisboa

,concelho,tipo_de_instalacao,nivel_de_tensao,ano,mes,cpes_quantidade
19578,Lisboa,Doméstico,Baixa Tensão Especial,2022,3,64
19579,Lisboa,Doméstico,Baixa Tensão Especial,2022,4,64
19580,Lisboa,Doméstico,Baixa Tensão Especial,2022,5,65
19581,Lisboa,Doméstico,Baixa Tensão Especial,2022,6,65
19582,Lisboa,Doméstico,Baixa Tensão Especial,2022,7,65
...,...,...,...,...,...,...
19803,Lisboa,Outros,"Muito Alta, Alta e Média Tensões",2022,4,16
19804,Lisboa,Outros,"Muito Alta, Alta e Média Tensões",2022,5,16
19805,Lisboa,Outros,"Muito Alta, Alta e Média Tensões",2022,6,32
19806,Lisboa,Outros,"Muito Alta, Alta e Média Tensões",2022,7,32


### 5. Guardar Ficheiro

In [10]:
file_path = data_path + 'treated/caracterizacao-pes-contrato-ativo.csv'

df_agrupado_caracterizacao.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

DataFrame guardado em  ../data/treated/caracterizacao-pes-contrato-ativo.csv
